# 1 - ML Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "Salary cap", "Season", "Outcome in wins", "Playoffs", "Wins", "Losses", "PCT", "FG",
    "FGA", "FG%", "3P", "3PA", "3P%","2P", "2PA", "2P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", 
    "AST", "STL", "BLK", "TOV", "PF", "PTS"
]

target = ["Playoffs"]

In [6]:
# Load the data
file_path = Path('Data/NBA_Salary.csv')
df = pd.read_csv(file_path)[:-2]
# df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

In [45]:
print(df.dtypes)

Salary cap      int64
Wins            int64
Losses          int64
PCT           float64
Postseason     object
FG            float64
FGA           float64
FG%           float64
3P            float64
3PA           float64
3P%           float64
2P            float64
2PA           float64
2P%           float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
dtype: object


# Split the Data into Training and Testing

In [10]:
# Create our features
X = pd.get_dummies(df, columns= ["Salary cap",
                                "PCT",
                                "FG%",
                                "3P%",
                                "2P%",
                                "FT%",
                                "TRB",
                                "AST",
                                "STL",
                                "BLK",
                                "TOV",
                                "PTS"]).drop('Playoffs', axis=1)

# Create our target

y = df['Playoffs']
X.head()

,Team,Season,Outcome in wins,Wins,Losses,FG,FGA,3P,3PA,2P,...,PTS_116.3,PTS_116.4,PTS_116.6,PTS_117.0,PTS_117.7,PTS_117.8,PTS_118.0,PTS_118.6,PTS_118.7,PTS_120.1
0,Golden State,2021/2022,0,53,29,40.5,86.4,14.3,39.4,26.2,...,0,0,0,0,0,0,0,0,0,0
1,Brooklyn,2021/2022,0,44,38,42.0,88.4,11.5,31.7,30.5,...,0,0,0,0,0,0,0,0,0,0
2,LA Clippers,2021/2022,0,42,40,40.1,87.4,12.8,34.2,27.3,...,0,0,0,0,0,0,0,0,0,0
3,LA Lakers,2021/2022,0,33,49,41.6,88.8,12.0,34.5,29.7,...,0,0,0,0,0,0,0,0,0,0
4,Milwaukee,2021/2022,0,51,31,41.8,89.4,14.1,38.4,27.8,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X.describe()

,Outcome in wins,Wins,Losses,FG,FGA,3P,3PA,2P,2PA,FT,...,PTS_116.3,PTS_116.4,PTS_116.6,PTS_117.0,PTS_117.7,PTS_117.8,PTS_118.0,PTS_118.6,PTS_118.7,PTS_120.1
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,...,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,2.209459,39.054054,38.608108,40.714189,88.120270,11.833108,32.962838,28.883108,55.154054,17.268243,...,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757
std,4.069554,11.675769,11.160935,1.608608,2.282795,1.681245,4.430495,2.141599,4.466982,1.529352,...,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
min,0.000000,15.000000,17.000000,36.700000,82.800000,8.000000,22.500000,23.100000,41.900000,12.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,31.000000,30.000000,39.400000,86.475000,10.675000,29.975000,27.375000,52.475000,16.275000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,41.500000,38.000000,40.700000,88.100000,11.600000,32.700000,29.000000,55.300000,17.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.250000,48.000000,46.000000,41.900000,89.625000,12.925000,35.950000,30.525000,58.300000,18.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,16.000000,65.000000,65.000000,44.700000,94.400000,16.700000,45.400000,33.400000,64.400000,22.600000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Check the balance of our target values
y.value_counts()

Yes    79
No     69
Name: Playoffs, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'No': 51, 'Yes': 60})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'No': 60, 'Yes': 60})

In [15]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

ValueError: could not convert string to float: 'LA Lakers'

In [52]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [23]:
# Display the confusion matrix

from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm1_df = pd.DataFrame(cm1, index=["Actual Missed Playoffs", "Actual Playoffs"], columns=["Predicted Missed Playoffs", "Predicted Playoffs"])
cm1_df

,Predicted Missed Playoffs,Predicted Playoffs
Actual Missed Playoffs,19,0
Actual Playoffs,0,18


In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



### SMOTE Oversampling

In [25]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'Missed Playoffs': 61, 'Playoffs': 61})

In [26]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [27]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [28]:
# Display the confusion matrix
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm2_df = pd.DataFrame(cm2, index=["Actual Missed Playoffs", "Actual Playoffs"], columns=["Predicted Missed Playoffs", "Predicted Playoffs"])
cm2_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,19,0
Actual low_risk,0,18


In [29]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Missed Playoffs': 50, 'Playoffs': 50})

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [33]:
# Display the confusion matrix
cm3 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm3_df = pd.DataFrame(cm1, index=["Actual Missed Playoffs", "Actual Playoffs"], columns=["Predicted Missed Playoffs", "Predicted Playoffs"])
cm3_df

,Predicted Missed Playoffs,Predicted Playoffs
Actual Missed Playoffs,19,0
Actual Playoffs,0,18


In [34]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [35]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Missed Playoffs': 50, 'Playoffs': 50})

In [36]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [38]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm1, index=["Actual Missed Playoffs", "Actual Playoffs"], columns=["Predicted Missed Playoffs", "Predicted Playoffs"])
cm_df

,Predicted Missed Playoffs,Predicted Playoffs
Actual Missed Playoffs,19,0
Actual Playoffs,0,18


In [39]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37

